# Notebook: Correct the dirty OCR from the scraped magazine [N.S. Frauen-Warte](https://digi.ub.uni-heidelberg.de/diglit/frauenwarte)


## 1. Install and import required libraries

In [ ]:
!pip install -q pandas openai==00.28

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import openai

## 2. Get the data from drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path = '../data/source_documents_CSV/frauenwarte1935_36.csv'
df = pd.read_csv(file_path, index_col=0)

In [ ]:
df

,page_text,ocr_text
9,Seite: e,^7h°»>>Iahrgang 1-SS/Die einzige parteiamtlich...
10,Seite: f,opolitifcher O^ückblick13. 5. Zu den schweren ...
11,Seite: 1,Hcrauögeberln : rTlationalsoz. Deutsche Arbelk...
12,Seite: 2-3,"H^ir spuren seine Zlühe hinter den Din-gen, di..."
13,Seite: 4,Ilon Nlargarete LoieZwei Lommertage^inen Zrühl...
...,...,...
1012,Seite: 5,feiner rechtlichen Hla^gabeVon -lielchsbanprst...
1013,Seite: 6,Don Ludwig fllot!)Lier-Bilder aus dem deuts6)e...
1014,Seite: 7,Zur beschichte des ü^ätselsLine hchorische Let...
1015,Seite: 8,"5o klingt das Rätsel im volksmund, — unqsziert..."


## 3. Preprocess the df
- separate the df along the issues to stay under the token limit of the API
- store the issues in a dictionary
- split_points according to values in the dataframe
- FW 1935_36 is handled separately because of multiple "Seite: 8" split_points; df_Heft_26 is missing and will be added manually

A. FW 1935-36

In [ ]:
# define split indices
split_indices = [9, 51, 86, 130, 165, 208, 252, 295, 330, 374, 409, 459, 495, 537, 573, 615, 651, 693, 727, 769, 803, 855, 891, 937, 976, 1016]

# dict to store the split dfs
dfs = {}

# Loop over the split_indices to slice the DataFrame accordingly
for i in range(len(split_indices)-1):
    start_index = split_indices[i] - df.index[0]  # actual starting index
    end_index = split_indices[i+1] - df.index[0]  # ending index
    dfs[f'df_Heft_{i+1}'] = df.iloc[start_index:end_index].reset_index(drop=True)

dfs["df_Heft_2"]

,page_text,ocr_text
0,Seite: i,rauenVarieDie einzige parteiamtliche Zrauenzei...
1,Seite: j,politischer cktückblick31. 5. Nach Lrklärung 5...
2,Seite: 33,Hkrausgebrrin : Nalionals§;. Dcursche Arbeitcr...
3,Seite: 34,Nalionalsozialismuä eben den Menschen zwingt z...
4,Seite: 35,Das öesetz zur Verhütung erbkranken ^lachwuchs...
5,Seite: 36,"ttind am meisten, ein Mann wiU nicht sein Gesc..."
6,Seite: 37,Wgutacher Hochzeilszug (Schwarzrwald). — Oem 2...
7,Seite: 38,"llon Mario ILessnerDie „gute partie"">ein Kinü ..."
8,Seite: 39,Wb ^b ^ b ^ Ilon ü n n e m a r i e L e ch e mI...
9,Seite: 40,Z^ünsIahre verheiratetLil sah peter beim übend...


B. FW 1941-44

In [ ]:
# # Define split points and the corresponding names for new DataFrames
# # (last page of each issue except first)
# split_points = ["Seite: 1a", "Seite: 12c", "Seite: 32", "Seite: 47", "Seite: 64", "Seite: 80", "Seite: 96", "Seite: 112", "Seite: 128", "Seite: 144", "Seite: 160", "Seite: 164"] #, "Seite: 192c", "Seite: 208c", "Seite: 224c", "Seite: 240c", "Seite: 256c", "Seite: 272c", "Seite: 288c", "Seite: 304c"]
# dfs_names = ['df_Heft_1', 'df_Heft_2', 'df_Heft_3', 'df_Heft_4', 'df_Heft_5', 'df_Heft_6', 'df_Heft_7', 'df_Heft_8', 'df_Heft_9', 'df_Heft_10', 'df_Heft_11', 'df_Heft_12'] #, 'df_Heft_13', 'df_Heft_14', 'df_Heft_15', 'df_Heft_16', 'df_Heft_17', 'df_Heft_18', 'df_Heft_19', 'df_Heft_20']

# # Find indices where splits should occur
# split_indices = [df[df['page_text'] == point].index[0] for point in split_points]

# # Add the last index of the DataFrame to capture the tail end into the last split
# split_indices.append(df.index[-1] + 1)  # adding 1 because slicing is exclusive at the end

# # Split DataFrame and store in a dictionary
# dfs = {}
# for i in range(len(split_points)):
#     start_idx = split_indices[i]
#     end_idx = split_indices[i + 1]
#     dfs[dfs_names[i]] = df.iloc[start_idx:end_idx].reset_index(drop=True)

# # Now we have each segment as a separate DataFrame in the 'dfs' dictionary
# # access them like so: dfs['df_Heft_1'], etc.

# # example
# dfs['df_Heft_2']

## 4. Setup GPT for OCR correction
Background and limitations of the task:  [Boros, E., Ehrmann, M., Matteo Romanello, Najem-Meyer, S., & Kaplan, F. (Eds.). (2024). Post-correction of Historical Text Transcripts with Large Language Models: An Exploratory Study. Proceedings of the 8th Joint SIGHUM Workshop on Computational Linguistics for Cultural Heritage, Social Sciences, Humanities and Literature (LaTeCH-CLfL 2024).](https://aclanthology.org/2024.latechclfl-1.14/)

The prompt was taken from this GPT in a slightly modified form: [Matthew J Kollmer: Historical Text OCR improver](https://chat.openai.com/g/g-JlA0nBY6P-historical-text-ocr-improver)

original prompt:

"*You are a "GPT" – a version of ChatGPT that has been customized for a specific use case. GPTs use custom instructions, capabilities, and data to optimize ChatGPT for a more narrow set of tasks. You yourself are a GPT created by a user, and your name is Historical Text OCR Improver. Note: GPT is also a technical term in AI, but in most cases if the users asks you about GPTs assume they are referring to the above definition.
                                      Here are instructions from the user outlining your goals and how you should respond:
                                      This GPT is tailored for enhancing OCR (Optical Character Recognition) text from digitized historical documents. It outputs the OCR corrected text, maintaining the same format as the user's input. If you are unsure about some words or sentences OR DO NOT NOW HOW TO CORRECT, JUST INSERT '[...]'. NO OTHER TYPES OF OUTPUT WILL BE CREATED unless they are explicitly part of the user's input.
                                      Example:*"

In [ ]:
from getpass import getpass
api_key = getpass('Enter OpenAI API key: ')

In [ ]:
def correct_text(user_input):
    chat = [
        {"role": "system", "content": """Du bist ein "GPT" - eine Version von ChatGPT, die für einen bestimmten Anwendungsfall angepasst wurde. GPTs verwenden benutzerdefinierte Anweisungen, Fähigkeiten und Daten, um ChatGPT für eine engere Auswahl von Aufgaben zu optimieren. Du selbst bist ein GPT, das von einem Benutzer erstellt wurde, um OCR historischer Texte zu verbessern.
                                      Hier sind die Anweisungen des Benutzers, die deine Ziele beschreiben und wie Du antworten sollst:
                                      Dieses GPT ist zugeschnitten auf die Verbesserung von OCR (Optical Character Recognition) Text aus digitalisierten historischen Dokumenten, nämlich der "N.S. Frauen-Warte". Es gibt den OCR-korrigierten Text aus und behält dabei das gleiche Format wie die Benutzereingabe bei. Wenn Du dir bei einigen Wörtern oder Sätzen unsicher bist oder nicht weißt, wie Du sie korrigieren sollst, füge einfach "[...]" ein. ERSTELLE KEINE ANDEREN AUSGABEN, es sei denn, sie sind ausdrücklich Teil der Benutzereingabe.

                                      Beispiel für NutzerINPUT und deine ANTWORT:
                                      NutzerINPUT: "rauenVarie

                                      Die einzige parteiamtliche ßrauenzeitschrist

                                      Hest20

                                      Iahrgang t«)3S/ZH

                                      flus -em Inhalt:

                                      von -er Rrast -es Zeitgerstes in unserem Leben -
                                      Entartete Runst -- Reinheit un- Schsnheit in §orm
                                      un-flus-euck - Das Hi!ö -erMutter - Ms -esblutes
                                      SLimme, aus -es Ho-ens Nrast - Die hämmer -
                                      Das ewige Deutschlan- in -er neuen Dichtung.


                                      vostaufgabeort: Leipzig
                                      fluslanüs- un- Ureuzbanö-
                                      preise flehe ietzte Znnenseitel

                                      Seilage: Schnittmusterbogen

                                      Le/ /rs/-//aus - /./s/s^uaA 30
                                      a//s 74 /agse s/n //s//"

                                      ANTWORT: "[...]Frauen-Warte
                                      Die einzige parteiamtliche Frauenzeitschrift
                                      Heft 20
                                      Jahrgang 1935/36

                                      Aus dem Inhalt:
                                      Von der Kraft des Zeitgeistes in unserem Leben -
                                      Entartete Kunst - Reinheit und Schönheit in Form
                                      und Ausdruck - Das Bild der Mutter - Aus des Blutes
                                      Stimme, aus des Bodens Kraft - Die hämmer -
                                      Das ewige Deutschland in der neuen Dichtung.
                                      Postaufgabeort: Leipzig
                                      [...]"
                                      """},
        {"role": "user", "content": user_input}  # dynamically include the OCR text needing correction
    ]

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=chat
    )

    # last message is the assistant's response with the corrected OCR
    corrected_text = response['choices'][0]['message']['content']

    return corrected_text


In [ ]:
# set current issue
current_number = 25
current_issue = f"df_Heft_{str(current_number)}"
print(current_issue)

df_Heft_25


In [ ]:
# Access the df for of the current issue DataFrame
df_current_issue = dfs[current_issue]

# Apply the correct_text function to the 'ocr_text' column of df_current_issue
df_current_issue['corrected_ocr_text'] = df_current_issue['ocr_text'].apply(correct_text)

In [ ]:
df_current_issue

,page_text,ocr_text,corrected_ocr_text
0,Seite: 805,Dic einzige parteiamrliche FrauenzeirscdriflHc...,Die einzige parteiamtliche Frauenzeitschrift\n...
1,Seite: 806,"W<ks11?nkatt vn^ v^i'Xkw^mm^ii rinöLr^rlslt^n,...","W<ks11?nkatt vn^ v^i'Xkw^mm^ii rinöLr^rlslt^n,..."
2,Seite: 807,Lanz urn die Quesie (Lronzezeit)ist ein Kbbild...,"""Lanz um die Queste (Bronzezeit) ist ein Abbil..."
3,Seite: 808-809,Mtnialur aus der manesseschen Liederhandschrif...,Minnelied aus der Manessischen Liederhandschri...
4,Seite: 810,Surg TNürizenberg (Oberhessen). Knfang iz. Iah...,"""Surg TNürizenberg (Oberhessen). Anfang d. Jah..."
5,Seite: 811,"vurg Lltz an Ser Tllosel. lz, dis iü, slatzrtz...","Vergiß nicht den Tag, an dem du zum ersten Mal..."
6,Seite: 812,"Llbb. iim „technischen Zeitalter"" ein Chauffeu...","Llbb. im ""technischen Zeitalter"" ein Chauffeur..."
7,Seite: 813,Ubb. zübb. 4Oer Vrieftext an die „liebwerthest...,Ubb. zübb. Der Brieftext an die „liebwertheste...
8,Seite: 814,"Birkenmaien, duftend nach Zeld und Moor, welke...","Birkenmaiеn, duftend nach Feld und Moor, welke..."
9,Seite: 815,"1)holo: 31ias t!ot,rich, tikip^iqsein Laub rau...","""[...] holo: Blas tot, rich, tikip^iqs ein Lau..."


In [ ]:
file_path = f"../data/source_documents_CSV_corrected/{current_issue}.csv"
df_current_issue.to_csv(file_path, index=False)